<a href="https://colab.research.google.com/github/DevanMayer/EE-480-Assignments/blob/main/ca3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EE 480 - Homework #3: Transmission Line Modeling & Compensation**

**Instructor:** Dr. Saeed Manshadi


### **Instructions**

This assignment focuses on the application of transmission line models (nominal-π and long-line), performance analysis (voltage regulation), and compensation techniques. All solutions must be calculated using **Julia** within this notebook.


**Submission:**
*   Please follow the submission instructions provided on Canvas to save your completed notebook back to your private GitHub repository.



---
### **Problem 1**

A 200-km, 230-kV, 60-Hz, three-phase line has a positive-sequence series impedance `z = 0.08 + j0.48` Ω/km and a positive-sequence shunt admittance `y = j3.33×10⁻⁶` S/km. At full load, the line delivers 250 MW at 0.99 p.f. lagging and at 220 kV. Using the **nominal π circuit**, calculate:
(a) the ABCD parameters,
(b) the sending-end voltage and current, and
(c) the percent voltage regulation.

**Guidance:**
*   This is a direct application of the medium-length line (nominal-π) model.
*   First, calculate the total series impedance `Z = z * l` and total shunt admittance `Y = y * l`.
*   Use the nominal-π formulas to find the ABCD parameters.
*   Calculate the receiving-end current `I_R` from the load's P, pf, and V. Remember to work in per-phase values.
*   Apply the two-port network equations to find `V_S` and `I_S`.
*   Use the voltage regulation formula: `VR% = (|Vs|/|A| - |Vr,fl|) / |Vr,fl| * 100`.

In [9]:
# --- Problem 1 Solution ---

# --- Given Data ---
len_km = 200.0
V_rated_LL = 230e3
f = 60.0
z_per_km = 0.08 + 0.48im
y_per_km = 3.33e-6im

# Load Data
P_R_3ph = 250e6
pf_R = 0.99 # lagging
V_R_LL = 220e3

# --- Step 1: Calculate Total Z and Y ---
Z_total = z_per_km*len_km
Y_total = y_per_km*len_km

println("--- Total Line Parameters ---")
println("Z = ", Z_total, " Ω")
println("Y = ", Y_total, " S")

# --- Step 2 (Part a): Calculate ABCD Parameters ---
A = (1+((Y_total*Z_total)/2))
B = Z_total
C = Y_total*(1+((Y_total*Z_total)/2))
D = A # For nominal-pi, D=A

println("\n--- Part (a): ABCD Parameters ---")
println("A = D = ", A)
println("B = ", B, " Ω")
println("C = ", C, " S")

# --- Step 3 (Part b): Calculate Sending-End Quantities ---
# First, find the receiving-end per-phase voltage and current
V_R_LN = V_R_LL / sqrt(3) + 0.0im # Reference phasor
S_R_3ph_mag = P_R_3ph / pf_R
phi_R = acos(pf_R) # Lagging, so angle is positive
S_R_3ph = S_R_3ph_mag * exp(im*phi_R)
I_R = conj( (S_R_3ph/3) / V_R_LN )

# Now find Vs and Is
V_S = A*V_R_LN+B*I_R
I_S = C*V_R_LN+D*I_R
P_S_3ph = 3 * real(V_S * conj(I_S))

println("\n--- Part (b): Sending-End Quantities ---")
println("Sending-End Voltage (Vs_LL) = ", abs(V_S) * sqrt(3), " V")
println("Sending-End Current (Is) = ", abs(I_S), " A")

# --- Step 4 (Part c): Calculate Voltage Regulation ---
V_R_nl = abs(V_S) / abs(A)
VR_percent = (abs(V_R_nl)-abs(V_R_LN))/abs(V_R_LN)*100


println("\n--- Part (c): Voltage Regulation ---")
println("Percent Voltage Regulation = ", VR_percent, "%")

--- Total Line Parameters ---
Z = 16.0 + 96.0im Ω
Y = 0.0 + 0.0006659999999999999im S

--- Part (a): ABCD Parameters ---
A = D = 0.968032 + 0.005327999999999999im
B = 16.0 + 96.0im Ω
C = -3.5484479999999994e-6 + 0.000644709312im S

--- Part (b): Sending-End Quantities ---
Sending-End Voltage (Vs_LL) = 269167.2208985764 V
Sending-End Current (Is) = 635.1742524368295 A

--- Part (c): Voltage Regulation ---
Percent Voltage Regulation = 26.38723061879035%


---
### **Problem 2 (Combined with original Problem 7)**

A 60-Hz, 230-mile, three-phase overhead transmission line has the series impedance and shunt admittance `z = 0.8431∠79.04°` Ω/mi and `y = 5.105×10⁻⁶∠90°` S/mi. The load at the receiving end is 125 MW at unity power factor and at 215 kV.

(a) Find the **exact** ABCD parameters of the line.
(b) Determine the voltage, current, and both real and reactive power at the sending end.
(c) Determine the percent voltage regulation of the line.
(d) Find the wavelength and velocity of propagation of the line.
(e) For a series capacitive compensation of 70% (split with 35% at each end), determine the new ABCD parameters.
(f) Comment on the relative change in the magnitude of the B parameter and the maximum power that can be transmitted when series compensated.

**Guidance:**
*   This is a **long line** problem (230 miles > 150 miles or 250 km). You must use the exact long-line model with hyperbolic functions.
*   **Part (a):** Calculate the propagation constant `γ = sqrt(z*y)` and characteristic impedance `Zc = sqrt(z/y)`. Then find `γl`. Use `A = D = cosh(γl)`, `B = Zc*sinh(γl)`, and `C = (1/Zc)*sinh(γl)`.
*   **Parts (b) & (c):** These are the same procedure as Problem 1, but using the exact ABCD parameters you just calculated.
*   **Part (d):** `γ = α + jβ`. The wavelength is `λ = 2π / β`. The velocity is `v = ω / β`.
*   **Part (e):** The new equivalent `B_eq` for a line with series capacitors at each end is `B_eq = A_cap*B_line + B_cap*D_line`, where `B_line` is the original B parameter and `B_cap` is the impedance of the capacitor at one end.
*   **Part (f):** Compare `|B_eq|` to `|B_line|`. Since `P_max` is inversely proportional to `|B|`, comment on how `P_max` changes.

In [ ]:
# --- Problem 2 Solution ---

# --- Given Data ---
len_mi = 230.0
f = 60.0
z_per_mi_polar = (0.8431, 79.04)
y_per_mi_polar = (5.105e-6, 90.0)

# Load Data
P_R_3ph = 125e6
pf_R = 1.0
V_R_LL = 215e3

# --- Step 1 (Part a): Calculate Exact ABCD Parameters ---
z = z_per_mi_polar[1] * exp(im*deg2rad(z_per_mi_polar[2]))
y = y_per_mi_polar[1] * exp(im*deg2rad(y_per_mi_polar[2]))

gamma = sqrt(z*y)
Zc = sqrt(z/y)
gamma_l = gamma*len_mi

A = cosh(gamma)
B = Zc*sinh(gamma)
C = (1/Zc)*sinh(gamma)
D = A

println("--- Part (a): Exact ABCD Parameters ---")
println("gamma*l = ", gamma_l)
println("Zc = ", Zc, " Ω")
println("A = D = ", A)
println("B = ", B, " Ω")
println("C = ", C, " S")

# --- Parts (b) and (c) would be solved similarly to Problem 1 ---
# [Student to fill in the code for Vs, Is, Ps, Qs, and VR%]

V_R_LN = V_R_LL / sqrt(3) + 0.0im # Reference phasor
S_R_3ph_mag = P_R_3ph / pf_R
phi_R = acos(pf_R) # Lagging, so angle is positive
S_R_3ph = S_R_3ph_mag * exp(im*phi_R)
I_R = conj( (S_R_3ph/3) / V_R_LN )

V_S = A*V_R_LN+B*I_R
I_S = C*V_R_LN+D*I_R
P_S_3ph = 3 * real(V_S * conj(I_S))

V_R_nl = abs(V_S) / abs(A)
VR_percent = (abs(V_R_nl)-abs(V_R_LN))/abs(V_R_LN)*100

# --- Step 2 (Part d): Wavelength and Velocity ---
alpha = real(gamma)
beta = imag(gamma)
omega = 2 * pi * f

wavelength_mi = 2π / beta
velocity_mi_s = omega / beta
mi_per_s_to_percent_c = 1 / 186282 * 100

println("\n--- Part (d): Propagation Characteristics ---")
println("Wavelength (λ) = ", wavelength_mi, " miles")
println("Velocity of Propagation (v) = ", velocity_mi_s, " miles/s")
println("Velocity is ~", velocity_mi_s * mi_per_s_to_percent_c, "% of the speed of light.")

# --- Step 3 (Part e): Series Compensation ---
comp_factor = 0.70
Z_line = B # For long lines, B is not equal to Z
Z_cap_total = -im * imag(Z_line) * comp_factor
Z_cap_each_end = Z_cap_total / 2

# New ABCD matrix is [Cap]*[Line]*[Cap]
# A_eq = A_line + C_line*Z_cap
# B_eq = B_line + (A_line+D_line)*Z_cap + C_line*Z_cap^2
# Simplified: B_eq = B + 2*A*Z_cap_each_end (approx)
# Let's use the matrix multiplication for accuracy
A_cap = 1.0; B_cap = Z_cap_each_end; C_cap = 0.0; D_cap = 1.0;
A_eq = A_cap*A*D_cap + B_cap*C*D_cap # This is just one element of the product
# The full product is complex. Let's focus on B_eq.
B_eq = A_cap*B*D_cap + A_cap*A*B_cap + B_cap*D*D_cap
println("\n--- Part (e): Compensated Parameters ---")
println("Original B = ", B, " Ω")
println("Equivalent B_eq = ", B_eq, " Ω")

# --- Step 4 (Part f): Comments ---
B_mag_change = (abs(B_eq) - abs(B)) / abs(B) * 100
P_max_change = (abs(B) / abs(B_eq) - 1) * 100

println("\n--- Part (f): Comments ---")
println("The magnitude of B changed by ", B_mag_change, "%.")
println("Since P_max is inversely proportional to |B|, the maximum power transfer capability increased by approximately ", P_max_change, "%.")

---
### **Problem 3**

A 500-km, 500-kV, 60-Hz, uncompensated three-phase line has a positive-sequence series impedance `z = 0.03 + j0.35` Ω/km and a positive-sequence shunt admittance `y = j4.4×10⁻⁶` S/km. Determine the **equivalent π circuit** for the line and compare it with the **nominal π circuit**.

**Guidance:**
*   This problem directly compares the medium-line approximation (nominal-π) with the exact long-line model (equivalent-π) for a line that is clearly in the "long" category (500 km).
*   **For the nominal-π circuit:** The series arm `Z_nom` is just the total impedance `z * l`, and the shunt arms are `Y_nom/2 = (y * l) / 2`.
*   **For the equivalent-π circuit:** You must first calculate the propagation constant `γ = sqrt(z*y)` and characteristic impedance `Zc = sqrt(z/y)`. Then, use the long-line formulas for the equivalent circuit parameters:
    *   `Z' = Zc * sinh(γl)`
    *   `Y'/2 = (1/Zc) * tanh(γl/2)`
*   Finally, compare the magnitudes of `Z_nom` vs. `Z'` and `Y_nom/2` vs. `Y'/2`.




In [ ]:

# --- Problem 3 Solution ---

# --- Given Data ---
len_km = 500.0
z_per_km = 0.03 + 0.35im
y_per_km = 4.4e-6im

# --- Step 1: Calculate Nominal-π Circuit Parameters ---
Z_nominal = ... # YOUR CODE HERE
Y_nominal_over_2 = ... # YOUR CODE HERE

println("--- Nominal-π Circuit Parameters ---")
println("Z_nominal = ", Z_nominal, " Ω")
println("Y_nominal/2 = ", Y_nominal_over_2, " S")

# --- Step 2: Calculate Equivalent-π Circuit Parameters ---
# First, find gamma and Zc
gamma = ... # YOUR CODE HERE
Zc = ... # YOUR CODE HERE
gamma_l = gamma * len_km

# Now, find Z' and Y'/2
Z_prime = ... # YOUR CODE HERE
Y_prime_over_2 = ... # YOUR CODE HERE

println("\n--- Equivalent-π Circuit Parameters ---")
println("Z' (equivalent) = ", Z_prime, " Ω")
println("Y'/2 (equivalent) = ", Y_prime_over_2, " S")

# --- Step 3: Compare the two models ---
percent_diff_Z = (abs(Z_nominal) - abs(Z_prime)) / abs(Z_prime) * 100
percent_diff_Y = (abs(Y_prime_over_2) - abs(Y_nominal_over_2)) / abs(Y_nominal_over_2) * 100

println("\n--- Comparison ---")
println("The nominal-π series impedance is ", round(percent_diff_Z, digits=1), "% larger than the exact equivalent.")
println("The equivalent-π shunt admittance is ", round(percent_diff_Y, digits=1), "% larger than the nominal.")
println("\nFor a 500km line, the difference is significant, highlighting the need for the long-line model.")


---
### **Problem 4**

A 350-km, 500-kV, 60-Hz, three-phase uncompensated line has a positive-sequence series reactance `x = 0.34` Ω/km and a positive-sequence shunt admittance `y = j4.5×10⁻⁶` S/km. **Neglecting losses (lossless line)**, calculate:
(a) the ABCD parameters,
(b) the wavelength λ of the line in kilometers,
(c) the surge impedance loading in MW,
(d) If rated line voltage is applied to both ends of the line, calculate the theoretical maximum real power that the line can deliver.

**Guidance:**
*   This is a **lossless** long line problem. Use the simplified ABCD parameters (`A = cos(βl)`, `B = jZc*sin(βl)`, etc.).
*   First, calculate `L` and `C` from `x` and `y`. Then find `Zc = sqrt(L/C)` and `β = ω*sqrt(LC)`.
*   Wavelength `λ = 2π / β`.
*   SIL is `V_rated_LL² / Zc`.
*   `P_max` for a lossless line is `(Vs*Vr) / |B|`.

In [ ]:
# --- Problem 4 Solution ---

# --- Given Data ---
len_km = 350.0
V_rated_LL = 500e3
f = 60.0
x_per_km = 0.34
y_per_km = 4.5e-6im

# --- Step 1: Calculate L, C, Zc, and beta ---
omega = 2 * pi * f
L_per_km = ... # YOUR CODE HERE
C_per_km = ... # YOUR CODE HERE
Zc = ... # YOUR CODE HERE
beta = ... # YOUR CODE HERE
beta_l = beta * len_km

println("--- Fundamental Lossless Parameters ---")
println("Surge Impedance (Zc) = ", Zc, " Ω")
println("Phase Constant (beta) = ", beta, " rad/km")

# --- Step 2 (Part a): Lossless ABCD Parameters ---
A = ... # YOUR CODE HERE
B = ... # YOUR CODE HERE
C = ... # YOUR CODE HERE
D = A

println("\n--- Part (a): ABCD Parameters ---")
println("A = D = ", A)
println("B = ", B, " Ω")
println("C = ", C, " S")

# --- Step 3 (Parts b and c): Wavelength and SIL ---
wavelength_km = ... # YOUR CODE HERE
SIL = ... # YOUR CODE HERE

println("\n--- Parts (b) & (c) ---")
println("Wavelength (λ) = ", wavelength_km, " km")
println("Surge Impedance Loading (SIL) = ", SIL/1e6, " MW")

# --- Step 4 (Part d): Maximum Power ---
Vs = V_rated_LL
Vr = V_rated_LL
P_max = ... # YOUR CODE HERE

println("\n--- Part (d): Maximum Power ---")
println("Theoretical Maximum Power (P_max) = ", P_max/1e6, " MW")


---
### **Problem 5**

A three-phase power of 4000 MW is to be transmitted through four identical 60-Hz overhead transmission lines over a distance of 300 km. Based on a preliminary design, the phase constant and surge impedance of each line are `β = 9.46×10⁻⁴` rad/km and `Zc = 343` Ω, respectively. Assuming `Vs = 1.0` per unit, `VR = 0.9` per unit, and a power angle `δ = 36.87°`, determine a suitable nominal voltage level in kV based on the practical line-loadability criteria.

**Guidance:**
*   This is a design problem. We need to work backwards from the power requirement to find the necessary voltage.
*   First, calculate the power that **one line** must carry.
*   Use the real power transfer equation for a lossless line: `P = (Vs * Vr / Zc) * sin(δ)`. Note that this formula gives power in watts if `Vs` and `Vr` are in volts. If `Vs` and `Vr` are in per-unit, it gives power in per-unit on some `S_base`. A more direct formula is `P = (V_LL_s * V_LL_r / Zc) * sin(δ)`.
*   The problem gives `Vs` and `Vr` in per-unit. This implies `Vs = 1.0 * V_nominal` and `Vr = 0.9 * V_nominal`.
*   Substitute these into the power equation. The only unknown will be `V_nominal`. Solve for `V_nominal`.
*   The result will be the required nominal line-to-line voltage in kV. Choose the closest standard voltage level (e.g., 230 kV, 345 kV, 500 kV, 765 kV).


In [ ]:
# --- Problem 5 Solution ---

# --- Given Data ---
P_total = 4000e6 # W
num_lines = 4
dist_km = 300.0
beta = 9.46e-4 # rad/km
Zc = 343.0    # Ω
Vs_pu = 1.0
Vr_pu = 0.9
delta_deg = 36.87

# --- Step 1: Calculate the power to be transmitted by a single line ---
P_per_line = ... # YOUR CODE HERE
println("Power per line = ", P_per_line / 1e6, " MW")

# --- Step 2: Set up the power transfer equation ---
# P_per_line = ( (Vs_pu * V_nominal) * (Vr_pu * V_nominal) / Zc ) * sin(delta)
# P_per_line = ( Vs_pu * Vr_pu / Zc ) * V_nominal² * sin(delta)
# We need to solve for V_nominal.
delta_rad = deg2rad(delta_deg)

# --- Step 3: Solve for the nominal voltage ---
# Rearrange the formula: V_nominal² = (P_per_line * Zc) / (Vs_pu * Vr_pu * sin(delta))
V_nominal_squared = ... # YOUR CODE HERE
V_nominal = ... # YOUR CODE HERE

println("\nCalculated nominal line-to-line voltage = ", V_nominal / 1000, " kV")

# --- Step 4: Choose a suitable standard voltage level ---
# Compare the calculated value to standard levels like 230, 345, 500, 765 kV.
println("\nBased on the calculation, the most suitable standard nominal voltage level would be 765 kV.")